In [155]:
%run input/Format.ipynb
import ROOT as root
from array import array
root.gErrorIgnoreLevel = root.kFatal
import multiprocess as mp
%jsroot on

In [156]:
iOption0 = 3
iFile = 50
iType = 1
sim = 0

In [157]:
N_files = 1
N_rg = 8
N_arm = 2
N_layer = 8
rebin = 3
if iOption0: rebin = 4
Min_Entries = 100
colors=[1,2,4,root.kGreen+2,root.kMagenta,root.kOrange+4,root.kGray,root.kCyan]
file_path="input/NEW/"
file_names=["hadrons_missalign_try4_rg0-8.root","hadrons_missalign_try6_rg0-8.root","hadrons_missalign_try7_rg0-8.root",\
            "hadrons_missalign_try8_rg0-8.root","hadrons_missalign_try9_rg0-8.root","hadrons_missalign_try10_rg0-8.root",\
            "hadrons_missalign_try11_rg0-8.root","hadrons_missalign_try12_rg0-8.root","hadrons_missalign_try13_rg0-8.root",\
            "missalign_hadron_try0_rg0-8.root","missalign_hadron_try1_rg0-8.root","missalign_hadron_try2_rg0-8.root",\
            "missalign_hadron_try3_rg0-8.root","missalign_hadron_try4_rg0-8.root","missalign_hadron_try5_rg0-8.root",\
            "missalign_hadron_try6_rg0-8.root","missalign_hadron_try7_rg0-8.root","missalign_hadron_try8_rg0-8.root",\
            "missalign_hadron_try9_rg0-8.root","missalign_hadron_try10_rg0-8.root","missalign_hadron_try11_rg0-8.root",\
            "missalign_hadron_try12_rg0-8.root","missalign_hadron_try13_rg0-8.root","missalign_hadron_try14_rg0-8.root",\
            "missalign_hadron_try15_rg0-8.root","missalign_hadron_try16_rg0-8.root","missalign_hadron_new0_rg0-8.root",\
            "missalign_hadron_new1_rg0-8.root","missalign_hadron_new2_rg0-8.root","missalign_hadron_new3_rg0-8.root",\
            "missalign_hadron_new4_rg0-8.root","missalign_hadron_new5_rg0-8.root","missalign_hadron_new6_rg0-8.root",\
            "missalign_hadron_new7_rg0-8.root","missalign_hadron_new8_rg0-8.root","missalign_hadron_new9_rg0-8.root",\
            "missalign_hadron_new10_rg0-8.root","missalign_hadron_new11_rg0-8.root","missalign_hadron_new12_rg0-8.root",\
            "missalign_hadron_new13_rg0-8.root","missalign_hadron_new14_rg0-8.root","missalign_hadron_new15_rg0-8.root",\
            "missalign_hadron_new16_rg0-8.root","missalign_hadron_new17_rg0-8.root","missalign_hadron_new18_rg0-8.root",\
            "missalign_hadron_new19_rg0-8.root","missalign_hadron_new20_rg0-8.root","missalign_hadron_new21_rg0-8.root",\
            "missalign_hadron_new22_rg0-8.root","missalign_hadron_new23_rg0-8.root","missalign_hadron_new24_rg0-8.root",\
            "missalign_hadron_new25_rg0-8.root"] 
#new 7 - only east, 8 - west 2 cm 1.5 gev, 9 - as 8 but 1 Gev; 10 - as 8 but 0.5;  11 -alg corrected;  12 - vtx moved; 13 - vtx and dc moved;  14 - standard
# 19 - sep charge, 20 - 1 iter, 21 - 2 iter, 22 - 3 iter 1.5 pt, 23 - 0.5 pt, 24 - 1.5 phiDC; 25 - 0.5
hist_select_3D_names = ["dphi_phi0_pt_hist","dthe_the0_pt_hist"]
if iType == 1: hist_select_3D_names = ["dphi_phi0_init_hist","dthe_the0_init_hist","dphi_the0_init_hist","dthe_phi0_init_hist"]
if iType == 2: hist_select_3D_names = ["dphi_phi0_corr_hist","dthe_the0_corr_hist","dphi_the0_corr_hist","dthe_phi0_corr_hist"]
iOptames=["phi","the","the","phi"]

full_path = file_path+file_names[iFile]
if sim: full_path="../sim/my-kek.root"

In [158]:
hists_read = []

print(full_path)
infile = root.TFile.Open(full_path, "read")
for ilayer in range(N_layer):
    hist_select_3D0 = []
    NNN = 2
    if iFile>3: NNN = 4
    for ihist in range(NNN):
        hist_select_3D0.append(infile.Get(hist_select_3D_names[ihist]+f"_{ilayer%4}"))
        hist_select_3D0[-1].SetDirectory(root.nullptr)
    hists_read.append(hist_select_3D0)

infile.Close()

input/NEW/missalign_hadron_new24_rg0-8.root


In [159]:
boards_tot = [[-0.099,0.099],[-0.099,0.099]]
boards_gaus = [[-0.051,0.052],[-0.051,0.052]]
boards_pol0 = [[-0.09,0.09],[-0.09,0.09]] 

In [160]:
def misalignment_fitter(InHist, ilayer, iarm, irg):
    N_gaus=1

    hist_phi_bin = InHist.ProjectionY("kek")

    ibinz0 = 1+int(ilayer/4)+iarm*2+irg*4
    ibinz1 = 1+int(ilayer/4)+iarm*2+irg*4

    tot_entries = dphi_porj = InHist.ProjectionX(f"dphi_tot_{ilayer}_{iarm}_{irg}", 1, 120, ibinz0, ibinz1).GetEntries()
    N_bins_X = InHist.ProjectionX(f"dphi_tot1_{ilayer}_{iarm}_{irg}", 1, 120, ibinz0, ibinz1).GetNbinsX()

    phi_local =  array( 'd' )
    phi_local_err =  array( 'd' )

    Mean_local =  array( 'd' )
    Sigma_local =  array( 'd' )
    Mean_local_err =  array( 'd' )
    Sigma_local_err =  array( 'd' )

    int_fg_local =  array( 'd' )
    int_bg_local =  array( 'd' )

    N_bins = hist_phi_bin.GetNbinsX()
    N_iter = int(N_bins/rebin)
 
    for iphi in range(N_iter):
        c1 = root.TCanvas(f"c1_{ilayer}_{iphi}_{iarm}_{irg}",f"c1_{ilayer}_{iphi}_{iarm}_{irg}",720,360)

        ibiny0 = iphi*rebin+1
        ibiny1 = (iphi+1)*rebin

        dphi_porj = InHist.ProjectionX(f"dphi_hist_{ilayer}_{iphi}_{iarm}_{irg}", ibiny0, ibiny1, ibinz0, ibinz1)

        if dphi_porj.GetEntries() < Min_Entries:
            continue

        phi = 0.5* (hist_phi_bin.GetBinCenter(ibiny0) + hist_phi_bin.GetBinCenter(ibiny1))
        phi_err = rebin*hist_phi_bin.GetBinWidth(ibiny0)*0.5

        dphi_porj.SetLineColor(1)

        par_tot = array( 'd', (3*N_gaus+3)*[2.] )
        g_gaus= root.TF1( f'g_gaus_{ilayer}_{iphi}_{iarm}_{irg}', 'gaus',  boards_gaus[iarm][0], boards_gaus[iarm][1])
        #g_gaus2= root.TF1( f'g_gaus_2_{ilayer}_{iphi}', 'gaus',  boards_fit2[icentr][0]+central_bean_prev, boards_fit2[icentr][1]+central_bean_prev)#[0]/[2]/sqrt(2*3.14159)*exp(-0.5*pow((x-[1])/[2],2))
        g_pol = root.TF1( f'g_pol_{ilayer}_{iphi}_{iarm}_{irg}', '[0]+[1]*x*(abs(x)/x-1)+[2]*x*(abs(x)/x+1)',  boards_pol0[iarm][0], boards_pol0[iarm][1])
        Format_Func(g_pol,3,2)

        total = root.TF1( f'total_{ilayer}_{iphi}_{iarm}_{irg}', 'gaus(0)+[3]+[4]*x*(abs(x)/x-1)+[5]*x*(abs(x)/x+1)',boards_tot[iarm][0], boards_tot[iarm][1])
        Format_Func(total,2,6)

        g_gaus.SetParameter(1,0.)
        g_gaus.SetParameter(2,boards_tot[iarm][1])

        dphi_porj.Draw()
        dphi_porj.Fit( g_gaus, 'QR')
        #dphi_porj.Fit( g_gaus2, 'QR+' )
        dphi_porj.Fit( g_pol, 'QR+' )

        par_gaus = g_gaus.GetParameters()
        par_pol = g_pol.GetParameters()

        par_tot[0], par_tot[1], par_tot[2] = par_gaus[0], par_gaus[1], par_gaus[2]
        par_tot[3], par_tot[4], par_tot[5] = par_pol[0], par_pol[1], par_pol[1]

        total.SetParameters( par_tot )
        #total.SetParLimits(1,boards_tot[iarm][0]/10,boards_tot[iarm][1]/10)
        #total.SetParLimits(2,boards_fit[iarm][1]/10,2)
        
        dphi_porj.Draw()
        dphi_porj.Fit( total, 'QR' )

        par_tot = total.GetParameters()

        g_gaus= root.TF1( f'g_gaus_4_{ilayer}_{iphi}_{iarm}_{irg}', 'gaus',  par_tot[1]-2*par_tot[2], par_tot[1]+2*par_tot[2])

        Format_Func(g_gaus,3,4)

        par_gaus[0], par_gaus[1], par_gaus[2] = par_tot[0], par_tot[1], par_tot[2]
        par_pol[0], par_pol[1], par_pol[2] = par_tot[3], par_tot[4], par_tot[5]
        g_gaus.SetParameters(par_gaus)
        g_pol.SetParameters(par_pol)

        g_gaus.Draw('same')
        g_pol.Draw('same')

        if abs(total.GetParameter(1))>0.15: continue

        phi_local.append(phi)
        phi_local_err.append(phi_err)

        Mean_local.append(total.GetParameter(1))
        Sigma_local.append(total.GetParameter(2))
        Mean_local_err.append(total.GetParError(1)+abs(total.GetParameter(1))*(dphi_porj.GetEntries()/tot_entries*N_bins_X)**(-0.5))
        Sigma_local_err.append(total.GetParError(2))


        int_fg_local.append(g_gaus.Integral(total.GetParameter(1)-2*total.GetParameter(2),total.GetParameter(1)+2*total.GetParameter(2)))
        int_bg_local.append(g_pol.Integral(total.GetParameter(1)-2*total.GetParameter(2),total.GetParameter(1)+2*total.GetParameter(2)))
            
        #m2_pos.Draw("same") m2_neg.Draw("same")

        c1.SaveAs(f"output/misalignment/{iOption0}/fit_{ilayer}_{irg}_{iarm}_{iphi}.png" )

    return [Mean_local, Mean_local_err, Sigma_local, Sigma_local_err,phi_local,phi_local_err, int_fg_local, int_bg_local]


In [161]:
def misalignment_layer_fitter(InHist, ilayer):
    out_hists = []
    for irg in range(N_rg):
        out_hists1 = []
        for iarm in range(N_arm):
            out_hists1.append(misalignment_fitter(InHist[ilayer][iOption0], ilayer, iarm, irg))
        out_hists.append(out_hists1)
    return out_hists

In [162]:
Ntr = N_layer

pool = mp.Pool(Ntr)
output_array = pool.starmap(misalignment_layer_fitter, [(hists_read, i) for i in range(Ntr)])
pool.close()

In [163]:
Means, Mean_errs, Sigmas, Sigma_errs, phi_arr, phi_arr_err, Int_FG, Int_BG = [], [], [], [], [], [], [], []
for ilayer in range(N_layer):
    Means1, Mean_errs1, Sigmas1, Sigma_errs1, phi_arr1, phi_arr_err1, Int_FG1, Int_BG1 = [], [], [], [], [], [], [], []
    for irg in range(N_rg):
        Means0, Mean_errs0, Sigmas0, Sigma_errs0, phi_arr0, phi_arr_err0, Int_FG0, Int_BG0 = [], [], [], [], [], [], [], []
        for iarm in range(N_arm):
            Means0.append(output_array[ilayer][irg][iarm][0])
            Mean_errs0.append(output_array[ilayer][irg][iarm][1])
            Sigmas0.append(output_array[ilayer][irg][iarm][2])
            Sigma_errs0.append(output_array[ilayer][irg][iarm][3])
            phi_arr0.append(output_array[ilayer][irg][iarm][4])
            phi_arr_err0.append(output_array[ilayer][irg][iarm][5])
            Int_FG0.append(output_array[ilayer][irg][iarm][6])
            Int_BG0.append(output_array[ilayer][irg][iarm][7])
        Means1.append(Means0)
        Mean_errs1.append(Mean_errs0)
        Sigmas1.append(Sigmas0)
        Sigma_errs1.append(Sigma_errs0)
        phi_arr1.append(phi_arr0)
        phi_arr_err1.append(phi_arr_err0)
        Int_FG1.append(Int_FG0)
        Int_BG1.append(Int_BG0)
    Means.append(Means1)
    Mean_errs.append(Mean_errs1)
    Sigmas.append(Sigmas1)
    Sigma_errs.append(Sigma_errs1)
    phi_arr.append(phi_arr1)
    phi_arr_err.append(phi_arr_err1)
    Int_FG.append(Int_FG1)
    Int_BG.append(Int_BG1)

In [164]:
types = ["emc_r_dphi_","emc_min_r_dphi_","emc_cnt_dphi_","emc_r_dz_","emc_min_r_dz_","emc_cnt_dz_"]
fit_functions = []

for ilayer in range(N_layer):
    fit_functions0 = []
    for irg in range(N_rg):
        fit_functions1 = []
        for iarm in range(N_arm):
            fit_functions1.append(root.TF1(f"fit_functions_sigma_{ilayer}_{irg}_{iarm}","[0]*sin(x)+[1]*cos(x)+[2]",0.5,5.0))
            fit_functions1[-1].SetParameter(0,0.001)
            fit_functions1[-1].SetParameter(1,0.001)
            fit_functions1[-1].SetParameter(2,0.001)
            fit_functions1[-1].SetLineColor(colors[ilayer])
        fit_functions0.append(fit_functions1)
    fit_functions.append(fit_functions0)

In [165]:
iarm = 0
h2=[]
c2 = root.TCanvas(f"c2_{iarm}",f"c2_{iarm}",1400,1800)
c2.Divide(2,4)
g_mean = [[[[]]*N_layer]*N_rg]*N_arm
min, max = -0.025, 0.015
x0, x1 = 1.5, 4
if iOption0>0 and iOption0<3: x0, x1 = 1., 2.
legends =[]
for irg in range(N_rg):
    c2.cd(irg+1)
    h2.append(Format_Pad_old( x0, x1, min+0.008, max, f"{iOptames[iOption0]}0 (rad)", f"#LT d{iOptames[iOption0]} #GT", 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1,f"h_{iarm}_{irg}"))
    h2[-1].Draw()
    legends.append(root.TLegend(0.25,0.5,0.85,0.9))
    legends[-1].SetFillColorAlpha(0,0)
    legends[-1].SetLineWidth(0)
    legends[-1].SetNColumns(2)
    for ilayer in range(0,N_layer):
        N_phi = len(phi_arr[ilayer][irg][iarm])
        if N_phi<1: continue
        #print(N_phi,len (phi_arr[ilayer][irg][iarm]), len(Means[ilayer][irg][iarm]),len(phi_arr_err[ilayer][irg][iarm]),len(Mean_errs[ilayer][irg][iarm]))
        g_mean[iarm][irg][ilayer].append(root.TGraphErrors(N_phi, phi_arr[ilayer][irg][iarm], Means[ilayer][irg][iarm],phi_arr_err[ilayer][irg][iarm],Mean_errs[ilayer][irg][iarm]))
        Format_Graph(g_mean[iarm][irg][ilayer][-1],20+int(ilayer/4),1, colors[ilayer], 3, colors[ilayer],1,1)
        g_mean[iarm][irg][ilayer][-1].Fit(fit_functions[ilayer][irg][iarm],"Q","")
        g_mean[iarm][irg][ilayer][-1].Draw('P')
        if ilayer<4: legends[-1].AddEntry(g_mean[iarm][irg][ilayer][-1],f"pos layer {ilayer}","p")
        else: legends[-1].AddEntry(g_mean[iarm][irg][ilayer][-1],f"neg layer {ilayer}","p")
    legends[-1].Draw()
c2.Draw()
c2.SaveAs(f"output/misalignment/mean_{iarm}_{iOption0}.png")

In [166]:
iarm = 1
h3=[]
c3 = root.TCanvas(f"c3_{iarm}",f"c3_{iarm}",1400,1800)
c3.Divide(2,4)
g_mean = [[[[]]*N_layer]*N_rg]*N_arm
min, max = -0.015, 0.015
x0, x1 = -1., 1.5
if iOption0>0 and iOption0<3: x0, x1 = 1., 2.
legends =[]
for irg in range(N_rg):
    c3.cd(irg+1)
    h3.append(Format_Pad_old( x0, x1, min+0.008, max,f"{iOptames[iOption0]}0 (rad)", f"#LT d{iOptames[iOption0]} #GT", 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1,f"h_{iarm}_{irg}"))
    h3[-1].Draw()
    root.gPad.SetLeftMargin(0.15)
    legends.append(root.TLegend(0.25,0.5,0.85,0.9))
    legends[-1].SetFillColorAlpha(0,0)
    legends[-1].SetLineWidth(0)
    legends[-1].SetNColumns(2)
    legends[-1].SetTextSize(0.06)
    for ilayer in range(0,N_layer):
        N_phi = len(phi_arr[ilayer][irg][iarm])
        if N_phi<1: continue
        #print(N_phi,len (phi_arr[ilayer][irg][iarm]), len(Means[ilayer][irg][iarm]),len(phi_arr_err[ilayer][irg][iarm]),len(Mean_errs[ilayer][irg][iarm]))
        g_mean[iarm][irg][ilayer].append(root.TGraphErrors(N_phi, phi_arr[ilayer][irg][iarm], Means[ilayer][irg][iarm],phi_arr_err[ilayer][irg][iarm],Mean_errs[ilayer][irg][iarm]))
        Format_Graph(g_mean[iarm][irg][ilayer][-1],20+int(ilayer/4),1, colors[ilayer], 3, colors[ilayer],1,1)
        g_mean[iarm][irg][ilayer][-1].Fit(fit_functions[ilayer][irg][iarm],"Q","")
        g_mean[iarm][irg][ilayer][-1].Draw('P')
        if ilayer<4: legends[-1].AddEntry(g_mean[iarm][irg][ilayer][-1],f"pos layer {ilayer}","p")
        else: legends[-1].AddEntry(g_mean[iarm][irg][ilayer][-1],f"neg layer {ilayer}","p")
    legends[-1].Draw()
c3.Draw()
c3.SaveAs(f"output/misalignment/mean_{iarm}_{iOption0}.png")

In [167]:
if True:
    ilayer = 2
    print(f"const float "+iOptames[iOption0]+f"_offset_params[{N_rg}][{N_arm}][2][3] = "+"{")
    for irg in range(N_rg):
        print("    {")
        for iarm in range(N_arm):
            print("        {")     
            print("            {"+f"{round(fit_functions[ilayer+0][irg][iarm].GetParameter(0),6)}, {round(fit_functions[ilayer+0][irg][iarm].GetParameter(1),6)}, {round(fit_functions[ilayer+0][irg][iarm].GetParameter(2),6)}"+"},")
            print("            {"+f"{round(fit_functions[ilayer+4][irg][iarm].GetParameter(0),6)}, {round(fit_functions[ilayer+4][irg][iarm].GetParameter(1),6)}, {round(fit_functions[ilayer+4][irg][iarm].GetParameter(2),6)}"+"},")
            print("        },")  
        print("    },")   
    print("};")



const float phi_offset_params[8][2][2][3] = {
    {
        {
            {-0.005928, -0.000366, -0.003986},
            {-0.006396, -0.000353, -0.005492},
        },
        {
            {0.00112, 8.4e-05, 0.008172},
            {0.002122, -0.001268, 0.006724},
        },
    },
    {
        {
            {-0.005972, 0.000268, -0.003444},
            {-0.006314, -0.000662, -0.005777},
        },
        {
            {0.001337, -0.000117, 0.008202},
            {0.002033, -0.001888, 0.007209},
        },
    },
    {
        {
            {-0.006267, 0.000648, -0.00316},
            {-0.006601, -0.000595, -0.005842},
        },
        {
            {0.001292, 0.000963, 0.007236},
            {0.001911, -0.002326, 0.007676},
        },
    },
    {
        {
            {-0.005518, -0.000249, -0.004065},
            {-0.005921, -0.001257, -0.006443},
        },
        {
            {0.001046, 0.000122, 0.008152},
            {0.001752, -0.001825, 0.007412},
        },
    },
    {


In [168]:
if False:
    print(f"const float "+iOptames[iOption0]+f"_offset_param0[{N_rg}][{N_arm}][2][4] = "+"{")
    for irg in range(N_rg):
        print("    {")
        for iarm in range(N_arm):
            print("        {")     
            print("            {"+f"{round(fit_functions[0][irg][iarm].GetParameter(0),6)}, {round(fit_functions[1][irg][iarm].GetParameter(0),6)}, {round(fit_functions[2][irg][iarm].GetParameter(0),6)}, {round(fit_functions[3][irg][iarm].GetParameter(0),6)}"+"},")
            print("            {"+f"{round(fit_functions[4][irg][iarm].GetParameter(0),6)}, {round(fit_functions[5][irg][iarm].GetParameter(0),6)}, {round(fit_functions[6][irg][iarm].GetParameter(0),6)}, {round(fit_functions[7][irg][iarm].GetParameter(0),6)}"+"},")
            print("        },")  
        print("    },")   
    print("};")
    print(f"const float "+iOptames[iOption0]+f"_offset_param1[{N_rg}][{N_arm}][2][4] = "+"{")
    for irg in range(N_rg):
        print("    {")
        for iarm in range(N_arm):
            print("        {")     
            print("            {"+f"{round(fit_functions[0][irg][iarm].GetParameter(1),6)}, {round(fit_functions[1][irg][iarm].GetParameter(1),6)}, {round(fit_functions[2][irg][iarm].GetParameter(1),6)}, {round(fit_functions[3][irg][iarm].GetParameter(1),6)}"+"},")
            print("            {"+f"{round(fit_functions[4][irg][iarm].GetParameter(1),6)}, {round(fit_functions[5][irg][iarm].GetParameter(1),6)}, {round(fit_functions[6][irg][iarm].GetParameter(1),6)}, {round(fit_functions[7][irg][iarm].GetParameter(1),6)}"+"},")
            print("        },")  
        print("    },")   
    print("};")
    print(f"const float "+iOptames[iOption0]+f"_offset_param2[{N_rg}][{N_arm}][2][4] = "+"{")
    for irg in range(N_rg):
        print("    {")
        for iarm in range(N_arm):
            print("        {")     
            print("            {"+f"{round(fit_functions[0][irg][iarm].GetParameter(2),6)}, {round(fit_functions[1][irg][iarm].GetParameter(2),6)}, {round(fit_functions[2][irg][iarm].GetParameter(2),6)}, {round(fit_functions[3][irg][iarm].GetParameter(2),6)}"+"},")
            print("            {"+f"{round(fit_functions[4][irg][iarm].GetParameter(2),6)}, {round(fit_functions[5][irg][iarm].GetParameter(2),6)}, {round(fit_functions[6][irg][iarm].GetParameter(2),6)}, {round(fit_functions[7][irg][iarm].GetParameter(2),6)}"+"},")
            print("        },")  
        print("    },")   
    print("};")     
     


In [169]:
if False:
    ilayer, ilayer1 = 1, 5
    if iOption0: print(f"const float the_offset_params[{N_rg}][{N_arm}][3] = "+"{")
    else: print(f"const float phi_offset_params[{N_rg}][{N_arm}][3] = "+"{")
    for irg in range(N_rg):
        print("    {")
        for iarm in range(N_arm):
            print("        {"+f"{round(fit_functions[ilayer][irg][iarm].GetParameter(0)+fit_functions[ilayer1][irg][iarm].GetParameter(0),6)}, \
    {round(fit_functions[ilayer][irg][iarm].GetParameter(1)+fit_functions[ilayer1][irg][iarm].GetParameter(1),6)}, \
    {round(fit_functions[ilayer][irg][iarm].GetParameter(2)+fit_functions[ilayer1][irg][iarm].GetParameter(2),6)}"+"},")
        print("    },")   
    print("};")        

In [170]:
iarm = 0
h4=[]
c4 = root.TCanvas(f"c4_{iarm}",f"c4_{iarm}",1400,1800)
c4.Divide(2,4)
g_mean = [[[[]]*N_layer]*N_rg]*N_arm
min, max = -0.015, 0.015
x0, x1 = 1.5, 4
if iOption0>0 and iOption0<3: x0, x1 = 1., 2.
legends =[]
for irg in range(N_rg):
    c4.cd(irg+1)
    h4.append(Format_Pad_old( x0, x1, 0, max, f"{iOptames[iOption0]}0 (rad)", f"sigma d{iOptames[iOption0]}", 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1,f"h_{iarm}_{irg}"))
    h4[-1].Draw()
    legends.append(root.TLegend(0.25,0.5,0.85,0.9))
    legends[-1].SetFillColorAlpha(0,0)
    legends[-1].SetLineWidth(0)
    legends[-1].SetNColumns(2)
    for ilayer in range(0,N_layer):
        N_phi = len(phi_arr[ilayer][irg][iarm])
        if N_phi<1: continue
        #print(N_phi,len (phi_arr[ilayer][irg][iarm]), len(Means[ilayer][irg][iarm]),len(phi_arr_err[ilayer][irg][iarm]),len(Mean_errs[ilayer][irg][iarm]))
        g_mean[iarm][irg][ilayer].append(root.TGraphErrors(N_phi, phi_arr[ilayer][irg][iarm], Sigmas[ilayer][irg][iarm],phi_arr_err[ilayer][irg][iarm],Sigma_errs[ilayer][irg][iarm]))
        Format_Graph(g_mean[iarm][irg][ilayer][-1],20+int(ilayer/4),1, colors[ilayer], 3, colors[ilayer],1,1)
        g_mean[iarm][irg][ilayer][-1].Fit("pol0","Q","")
        g_mean[iarm][irg][ilayer][-1].GetFunction("pol0").SetLineColor(colors[ilayer])
        g_mean[iarm][irg][ilayer][-1].Draw('P')
        if ilayer<4: legends[-1].AddEntry(g_mean[iarm][irg][ilayer][-1],f"pos layer {ilayer}","p")
        else: legends[-1].AddEntry(g_mean[iarm][irg][ilayer][-1],f"neg layer {ilayer}","p")
    legends[-1].Draw()
c4.Draw()
c4.SaveAs(f"output/misalignment/sigma_{iarm}_{iOption0}.png")

In [171]:
iarm = 1
h4=[]
c4 = root.TCanvas(f"c4_{iarm}",f"c4_{iarm}",1400,1800)
c4.Divide(2,4)
g_mean = [[[[]]*N_layer]*N_rg]*N_arm
min, max = -0.015, 0.015
x0, x1 = -1., 1.5
if iOption0>0 and iOption0<3: x0, x1 = 1., 2.
legends =[]
for irg in range(N_rg):
    c4.cd(irg+1)
    h4.append(Format_Pad_old( x0, x1, 0, max, f"{iOptames[iOption0]}0 (rad)", f"sigma d{iOptames[iOption0]}", 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1,f"h_{iarm}_{irg}"))
    h4[-1].Draw()
    legends.append(root.TLegend(0.25,0.5,0.85,0.9))
    legends[-1].SetFillColorAlpha(0,0)
    legends[-1].SetLineWidth(0)
    legends[-1].SetNColumns(2)
    for ilayer in range(0,N_layer):
        N_phi = len(phi_arr[ilayer][irg][iarm])
        if N_phi<1: continue
        #print(N_phi,len (phi_arr[ilayer][irg][iarm]), len(Means[ilayer][irg][iarm]),len(phi_arr_err[ilayer][irg][iarm]),len(Mean_errs[ilayer][irg][iarm]))
        g_mean[iarm][irg][ilayer].append(root.TGraphErrors(N_phi, phi_arr[ilayer][irg][iarm], Sigmas[ilayer][irg][iarm],phi_arr_err[ilayer][irg][iarm],Sigma_errs[ilayer][irg][iarm]))
        Format_Graph(g_mean[iarm][irg][ilayer][-1],20+int(ilayer/4),1, colors[ilayer], 3, colors[ilayer],1,1)
        g_mean[iarm][irg][ilayer][-1].Fit("pol0","Q","")
        g_mean[iarm][irg][ilayer][-1].GetFunction("pol0").SetLineColor(colors[ilayer])
        g_mean[iarm][irg][ilayer][-1].Draw('P')
        if ilayer<4: legends[-1].AddEntry(g_mean[iarm][irg][ilayer][-1],f"pos layer {ilayer}","p")
        else: legends[-1].AddEntry(g_mean[iarm][irg][ilayer][-1],f"neg layer {ilayer}","p")
    legends[-1].Draw()
c4.Draw()
c4.SaveAs(f"output/misalignment/sigma_{iarm}_{iOption0}.png")